In [7]:
import pandas as pd
import numpy as np
import os
import math
import datetime

In [8]:
file_name_affix = "final车场信息_上海停车_总车位_allMerged_time_type"
file_name_list = [ i for i in os.listdir(".") if i.startswith(file_name_affix)]
merge_type1_list = [i for i in os.listdir('.') if i.startswith("final车场信息_上海停车_总车位_allMerged_type1_2021")]
merge_type2_list = [i for i in os.listdir('.') if i.startswith("final车场信息_上海停车_总车位_allMerged_type2_2021")]
merge_type1_time_list = set([ i[i.find(".")+1:i.rfind(".")] for i in merge_type1_list ])
merge_type2_time_list = set([ i[i.find(".")+1:i.rfind(".")] for i in merge_type2_list ])
print(merge_type1_list, merge_type2_list)

['final车场信息_上海停车_总车位_allMerged_type1_2021.03.25.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.03.26.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.03.29.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.03.30.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.03.31.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.04.01.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.04.02.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.04.03.csv', 'final车场信息_上海停车_总车位_allMerged_type1_2021.04.04.csv'] ['final车场信息_上海停车_总车位_allMerged_type2_2021.03.25.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.03.26.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.03.27.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.03.29.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.03.30.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.03.31.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.04.01.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.04.02.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.04.03.csv', 'final车场信息_上海停车_总车位_allMerged_type2_2021.04.

In [9]:
basic_all_merge_byDay_path = ["final车场信息_上海停车_总车位_allMerged_timeAggregated_type1.csv", "final车场信息_上海停车_总车位_allMerged_timeAggregated_type2.csv"]
def getMerged(merge_name_list, merge_time_list, basic_all_merge_byDay_path, type):

    def cutTime(id_df, time):
        '''
            每一个id_df 是表示某一个id下所爬取的时间
            
        '''
        #print("id_df: ", id_df)
        id_df["starttime"] = pd.to_datetime(id_df["starttime"])
        # 根据每五分钟进行分割
        id_df["starttime"] = id_df["starttime"].apply(lambda x: datetime.datetime(x.year, x.month, x.day, x.hour, x.minute//5*5))
        # 截取7点 到 10点 的数据
        # index 是表示数据位置的索引
        index = id_df["starttime"].apply(lambda x: True if (x.hour > 6 and x.hour < 22) else False)
        #print("index: ", index)
        
        c = id_df[index]
        c = c.reset_index(drop=True)
        d = pd.DataFrame()
        #print("c: \n", c)
        if (len(c) != 0):
            year = c.iloc[0]["starttime"].year
            month = c.iloc[0]["starttime"].month
            day = c.iloc[0]["starttime"].day
            d[time] = pd.date_range(datetime.datetime(year, month, day, 7), datetime.datetime(year, month, day, 22), freq="5min")
            l = len(d[d[time].isin(c["starttime"])])
            d.loc[d[time].isin(c["starttime"]),  ["PERCENTAGE"]] = c["PERCENTAGE"][:l].to_numpy()
            d["PARKING_ID2"] = [str(id_df["PARKING_ID"].iloc[0])+ "_" + str(i) for i in range(len(d))]
            d[time] = d["PERCENTAGE"]
            #print("d: ", d)
            d = d.drop("PERCENTAGE", axis=1)
            #print("finally d: ", d)
            return d
        else:
            d = pd.DataFrame()
            return d



    # 对每个时间段的文件进行处理
    for i in merge_time_list:
        # 计算符合每个时间的文件
        remain = [ k for k in merge_name_list if k.startswith("final车场信息_上海停车_总车位_allMerged_type{type}_2021.{a}".format(type=type, a=i))]

        base_path = basic_all_merge_byDay_path
        for j in remain:
            if base_path in os.listdir('.'):
               data = pd.read_csv(j)
               data = data.sort_values(["PARKING_ID", "starttime"], axis=0)
               newData = data.groupby("PARKING_ID").apply(cutTime, time=i)
               base_data = pd.read_csv(base_path)
               #base_data.index = base_data.index.droplevel()
               print("newData: ", newData)
               # 注意由于版本问题，
               #newData.index = newData.index.droplevel()
               #df = pd.concat([base_data, newData], axis=1)
               df = pd.merge(base_data, newData ,how="outer", on="PARKING_ID2", validate="one_to_one")
               df = df.sort_index(axis=1)
               df.to_csv(base_path, index = False)
            else:
               data = pd.read_csv(j)
               # 按照名字与时间点进行排序
               data = data.sort_values(["PARKING_ID", "starttime"], axis=0)

               # 利用apply 对每个id对应的数据进行操作，返回进行切割后的数据
               # 并将每一个数据进行拼接
               newData = data.groupby("PARKING_ID").apply(cutTime, time=i)
               newData = newData.sort_index(axis=1)
               newData.to_csv(base_path, index = False)


In [10]:
getMerged(merge_type1_list, merge_type1_time_list, basic_all_merge_byDay_path[0],1)

newData:                     04.04        PARKING_ID2
PARKING_ID                                 
cm31023000001 0     0.10    cm31023000001_0
              1     0.10    cm31023000001_1
              2     0.10    cm31023000001_2
              3     0.10    cm31023000001_3
              4     0.10    cm31023000001_4
...                  ...                ...
xh31010400483 176    NaN  xh31010400483_176
              177    NaN  xh31010400483_177
              178   0.93  xh31010400483_178
              179    NaN  xh31010400483_179
              180    NaN  xh31010400483_180

[139189 rows x 2 columns]
newData:                     03.31        PARKING_ID2
PARKING_ID                                 
cm31023000001 0     0.10    cm31023000001_0
              1     0.10    cm31023000001_1
              2     0.10    cm31023000001_2
              3     0.10    cm31023000001_3
              4      NaN    cm31023000001_4
...                  ...                ...
xh31010400483 176   0.94  xh3

In [11]:
getMerged(merge_type2_list, merge_type2_time_list, basic_all_merge_byDay_path[1], 2)

newData:                  04.01  PARKING_ID2
PARKING_ID                        
CM-002     0      NaN     CM-002_0
           1      NaN     CM-002_1
           2      NaN     CM-002_2
           3      NaN     CM-002_3
           4      NaN     CM-002_4
...               ...          ...
XHZ-026    176    NaN  XHZ-026_176
           177    NaN  XHZ-026_177
           178    NaN  XHZ-026_178
           179   0.46  XHZ-026_179
           180    NaN  XHZ-026_180

[76020 rows x 2 columns]
newData:                  04.04  PARKING_ID2
PARKING_ID                        
CM-002     0      1.0     CM-002_0
           1      1.0     CM-002_1
           2      1.0     CM-002_2
           3      1.0     CM-002_3
           4      1.0     CM-002_4
...               ...          ...
XHZ-026    176    NaN  XHZ-026_176
           177    NaN  XHZ-026_177
           178    NaN  XHZ-026_178
           179    NaN  XHZ-026_179
           180    NaN  XHZ-026_180

[75477 rows x 2 columns]
newData:          

In [12]:
for i in basic_all_merge_byDay_path:
    data = pd.read_csv(i)
    data = data.sort_index(axis=1)
    #print(data[data.isnull().T.any()].fillna(data[data.isnull().T.any()].mean(), axis=0))
    #print(data)
    #data_numpy = data.to_numpy()
    #data = data.fillna(-111)
    #data = data.replace(-111, np.nan)
    for k in range(len(data)):
        d = data.iloc[k]
        #print(math.isnan(d[0]), d[0] ,d[1])
        if (d.isnull().any()):
            for j in d.index:
                m = d[:-1].mean()
                #print("d[j]: ", d[j])
                #print(type(d[j]), d[j], type(j))
                if ((j != "PARKING_ID2") and (math.isnan(d[j]))) :
                    #print(type(d[j]))
                    d[j] = m
                    #print("d[j]2: ", d[j])
            data.iloc[k] = d
            #print("d2: ", d)
    print(data)
    data = data.fillna(method='ffill', axis=0)
    data = data.sort_index(axis=1)
    data.to_csv(i, index=False, mode="w")      

<ipython-input-12-5f4d32fdd6c7>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[j] = m
        03.25  03.26  03.29  03.30  03.31  04.01  04.02  04.03  04.04  \
0       0.224   0.10  0.224   0.10   0.10  0.224  0.224   0.72   0.10   
1       0.224   0.10  0.224   0.10   0.10  0.224  0.224   0.72   0.10   
2       0.224   0.10  0.224   0.10   0.10  0.224  0.224   0.72   0.10   
3       0.224   0.10  0.224   0.10   0.10  0.224  0.224   0.72   0.10   
4       0.100   0.10  0.100   0.10   0.10  0.100  0.100   0.10   0.10   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...   
141356  0.930   0.93  0.930   0.93   0.93  0.930  0.930   0.93   0.93   
141357    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
141358  0.930   0.93  0.930   0.93   0.93  0.930  0.930   0.93   